In [1]:
from alignment_calculation import loanbookMaker
from alignment_calculation import alignmentCalculator
import pandas as pd

%load_ext autoreload
%autoreload 2

In [2]:
lm = loanbookMaker()

In [ ]:
year = 2023
month = 12
start_year = 2022
start_month = 1
jst_codes = []

In [ ]:
loans = lm.make_loanbook(loan_year=year, month=month, additional_columns={'inst.prps':'MAX', 'dbtr.indstry_pd_wghtd_avrg':'AVG'})

In [ ]:
ac = alignmentCalculator(loan_file=loans,
                         custom_settings={'MAIN_PACTA_FILE'  : {2023: [r'D:\pacta\data\2024-02-14_AI_2023Q4_ECB-Company-Analytics.csv',
                                                                       r'D:\pacta\data\2024-02-14_AI_2023Q4_ECB-Company-Indicators.csv'],
                                                                2022: [r'D:\pacta\data\2023-02-17_AI_2022Q4_ECB PACTA for Banks Free dataset_EO_Company_Analytics.csv',
                                                                       r'D:\pacta\data\2023-02-17_AI_2022Q4_ECB PACTA for Banks Free dataset_EO_Company_Indicators.csv'],}
                                         })

In [ ]:
results = ac._preprocess_data(use_loan_file=True, 
                              individual_loans=False, 
                              loan_indicator='outstanding_amount', 
                              only_parents=True, 
                              facet_col=[], 
                              year=2023, 
                              use_region_file=True, 
                              normalise_method='bank')
pacta_data = ac._combine_asset_locations(year=2023)

In [ ]:
results = results.merge(
            results.groupby(['jst_code', 'dbtr_id', 'company_name', 'company_id', 'loan_id', 'name_company_y', 'sector', 'year'], as_index=False)['production'].sum(),
            how='left',
            left_on=['jst_code', 'dbtr_id', 'company_name', 'company_id', 'loan_id', 'name_company_y', 'sector', 'year'],
            right_on=['jst_code', 'dbtr_id', 'company_name', 'company_id', 'loan_id', 'name_company_y', 'sector', 'year'],
            suffixes=['','_total'])
results['tech_share'] = results['production']/results['production_total']

In [ ]:
temp = ac.calculate_net_alignment(bopo_split=True, facet_col=['company_id'])
temp

In [ ]:
ac.group_scores(temp, ['jst_code'])